In [14]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from PyALE import ale
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from tqdm import tqdm 

In [15]:
file_path = r"ModellingData_V3_PBE_ABE_NE.xlsx"
data = pd.read_excel(file_path)
data.head()

,GEOID,CT_5digit,bphigh,chd,diabetes,highchol,obesity,ABE1,ABE2,ABE3,ABE4,ABE5,ABE6,PBE1,PBE2,NE1,NE2,NE3
0,6037265301,265301,5.761042,0.500960,1.085414,6.762963,11.188110,-0.660764,0.692240,-0.394954,0.925184,-0.829787,0.235442,2.640152,-1.443477,0.059821,-0.053236,15.788235
1,6037401901,401901,16.223067,1.520913,2.788340,21.292775,35.741446,-0.316248,0.295115,-0.205460,0.128051,-0.506879,0.074576,2.905541,-1.458956,1.456775,-0.064177,18.570540
2,6037206020,206020,25.371634,3.241310,7.935621,23.024478,33.083716,-0.378863,0.253378,1.532130,3.858522,-0.640391,-0.504878,-0.537406,0.695641,2.116515,0.113467,18.510373
3,6037265303,265303,24.049650,3.103181,6.206362,23.079906,37.044221,0.275943,4.444663,-0.139991,-0.862088,-0.172616,-0.250905,2.788220,-1.508503,-0.018885,-0.016381,15.746365
4,6037222700,222700,28.248588,3.193318,5.895358,23.581431,49.864896,-0.465064,0.872278,0.292469,1.244664,-0.498661,0.033562,0.603629,-0.112641,1.575785,0.445801,17.049678


In [16]:
data.columns

Index(['GEOID', 'CT_5digit', 'bphigh', 'chd', 'diabetes', 'highchol',
       'obesity', 'ABE1', 'ABE2', 'ABE3', 'ABE4', 'ABE5', 'ABE6', 'PBE1',
       'PBE2', 'NE1', 'NE2', 'NE3'],
      dtype='object')

In [17]:
# Convert relevant object columns to numeric
columns_to_convert = [
    'bphigh', 'chd', 'diabetes', 'highchol', 'obesity',
    'ABE1', 'ABE2', 'ABE3', 'ABE4', 'ABE5', 'ABE6',
    'PBE1', 'PBE2',
    'NE1', 'NE2', 'NE3'
]

data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Select only the columns with float64 data type
float_cols = data.select_dtypes(include=['float64'])

# Fill missing values in float64 columns with the mean of each column
float_cols.fillna(float_cols.mean(), inplace=True)

# Replace the original columns in the DataFrame with the imputed columns
data[float_cols.columns] = float_cols

# Verify that missing values have been filled
print(data.isnull().sum())

GEOID        0
CT_5digit    0
bphigh       0
chd          0
diabetes     0
highchol     0
obesity      0
ABE1         0
ABE2         0
ABE3         0
ABE4         0
ABE5         0
ABE6         0
PBE1         0
PBE2         0
NE1          0
NE2          0
NE3          0
dtype: int64


In [18]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from shapely.geometry import Point


shapefile = gpd.read_file(r'C:\Users\24652\Desktop\Study area\CENSUS_Tracts.shp')  

In [19]:
# Ensure geoid column names match between the two datasets
data['GEOID'] = data['GEOID'].astype(str)
shapefile['GEOID'] = shapefile['GEOID'].astype(str).str[1:]

In [20]:
# Merge CSV and Shapefile on `geoid`
gdf = shapefile.merge(data, on='GEOID', how='right')

In [25]:
# gdf = ...  # your GeoDataFrame already loaded (projected CRS!)
DISEASE_COLS = ['bphigh', 'chd', 'diabetes', 'highchol', 'obesity']
GROUP_COL = 'city'                 # or None to summarize all together
FEATURE_COLS = [c for c in gdf.columns
                if c not in {'geometry','GEOID',GROUP_COL,*DISEASE_COLS}]

BUFFER_SIZE = 1.0                  # in units of your CRS
MIN_NEIGHBORS = 20
RF_PARAMS = dict(n_estimators=500, max_features=6, random_state=42, n_jobs=-1)

KEEP_GEOID_IN_PRED_CSV = False     # keep GEOID in per-row file
OUTPUT_DIR = './'                  # where to write CSVs

# AIC for RF (use your paper’s k). Default: p + 1 (features + intercept).
AIC_K = len(FEATURE_COLS) + 1
# ----------------------------------------------------

In [28]:
def _compute_local_metrics(y_true, y_pred, k=AIC_K):
    """Local-only metrics matching the paper’s equations."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    y_true = y_true[mask]; y_pred = y_pred[mask]
    n = y_true.size
    if n == 0:
        return dict(n=0, res_min=np.nan, res_max=np.nan, res_mean=np.nan,
                    rmse=np.nan, mae=np.nan, r2=np.nan, aic=np.nan)
    resid = y_true - y_pred
    rss = float(np.sum(resid**2))
    rmse = float(np.sqrt(rss / n))
    mae = float(mean_absolute_error(y_true, y_pred))
    # R^2 (manual to avoid edge cases)
    r2 = np.nan
    if n >= 2:
        sst = float(np.sum((y_true - y_true.mean())**2))
        if sst > 0:
            r2 = 1.0 - rss / sst
    aic = float(n * np.log(rss / max(n,1)) + 2 * k)
    return dict(
        n=int(n),
        res_min=float(np.min(resid)),
        res_max=float(np.max(resid)),
        res_mean=float(np.mean(resid)),
        rmse=rmse, mae=mae, r2=r2, aic=aic
    )


def _local_rf_per_row(gdf: gpd.GeoDataFrame, target_col: str):
    """
    Fit a local RF for each row using neighbors within BUFFER_SIZE.
    Returns a DataFrame with: yhat_local, loc_R2, and per-feature importances (wide).
    """
    # Safety: must have projected CRS (units in meters/km/etc.)
    assert gdf.crs is not None, "gdf.crs is None; reproject to a projected CRS before running."
    gdf = gdf.reset_index(drop=True)
    X_all = gdf[FEATURE_COLS].to_numpy()
    y_all = gdf[target_col].to_numpy()

    # Build spatial index (lazy)
    _ = gdf.sindex
    sidx = gdf.sindex
    geoms = gdf.geometry.values

    yhat_local = np.full(len(gdf), np.nan, float)
    loc_r2     = np.full(len(gdf), np.nan, float)
    imps       = np.full((len(gdf), len(FEATURE_COLS)), np.nan, float)

    for i in tqdm(range(len(gdf)), desc=f"Local RF [{target_col}]"):
        gi = geoms[i]
        if gi is None or gi.is_empty:
            continue

        # ✅ query with GEOMETRY, not bbox tuple
        buf_geom = gi.buffer(BUFFER_SIZE)
        cand_idx = list(sidx.query(buf_geom, predicate='intersects'))
        if not cand_idx:
            continue

        cand  = gdf.iloc[cand_idx]
        # precise distance cutoff
        neigh = cand.loc[cand.geometry.distance(gi) <= BUFFER_SIZE]
        if len(neigh) < MIN_NEIGHBORS:
            continue

        Xn = neigh[FEATURE_COLS].to_numpy()
        yn = neigh[target_col].to_numpy()

        rf = RandomForestRegressor(**RF_PARAMS)
        rf.fit(Xn, yn)

        # predict focal row
        yhat_local[i] = rf.predict(X_all[i:i+1])[0]

        # local fit R^2 on neighborhood
        yhat_n = rf.predict(Xn)
        sst = float(np.sum((yn - yn.mean())**2))
        if sst > 0:
            loc_r2[i] = 1.0 - float(np.sum((yn - yhat_n)**2)) / sst

        # feature importances
        try:
            imps[i, :] = rf.feature_importances_
        except Exception:
            pass

    out = pd.DataFrame({
        'yhat_local': yhat_local,
        'loc_R2': loc_r2
    })
    for j, f in enumerate(FEATURE_COLS):
        out[f'importance_{f}'] = imps[:, j]
    return out


def _summarize_local_by_group(df: pd.DataFrame, disease: str):
    """Compute LOCAL metrics per group (or overall if GROUP_COL is None)."""
    rows = []
    groups = ['All'] if (GROUP_COL is None or GROUP_COL not in df.columns) else sorted(df[GROUP_COL].dropna().unique())
    for g in groups:
        sub = df if GROUP_COL is None or GROUP_COL not in df.columns else df[df[GROUP_COL] == g]
        m = _compute_local_metrics(sub[disease].values, sub[f'yhat_local_{disease}'].values, k=AIC_K)
        rows.append({'disease': disease, 'group': g, **m})
    return pd.DataFrame(rows)



In [29]:
# =========================
# RUN (LOCAL ONLY)
# =========================
all_metric_frames = []
for dz in DISEASE_COLS:
    print(f"\n=== Disease: {dz} (LOCAL) ===")
    local_out = _local_rf_per_row(gdf, target_col=dz)

    # attach per-row outputs
    gdf[f'yhat_local_{dz}'] = local_out['yhat_local'].values
    gdf[f'loc_R2_{dz}']     = local_out['loc_R2'].values
    # (optionally keep importances per feature—already in `local_out`)

    # ---- write per-row predictions (lean) ----
    per_row_cols = []
    if KEEP_GEOID_IN_PRED_CSV and 'GEOID' in gdf.columns:
        per_row_cols.append('GEOID')
    if GROUP_COL in gdf.columns:
        per_row_cols.append(GROUP_COL)
    per_row_cols += [dz, f'yhat_local_{dz}', f'loc_R2_{dz}']
    pred_df = gdf[per_row_cols].copy()
    pred_path = f"{OUTPUT_DIR}{dz}_local_predictions.csv"
    pred_df.to_csv(pred_path, index=False)

    # ---- compute + store tidy LOCAL metrics by group ----
    metrics_df = _summarize_local_by_group(gdf, disease=dz)
    metrics_path = f"{OUTPUT_DIR}{dz}_local_metrics_tidy.csv"
    metrics_df.to_csv(metrics_path, index=False)

    print(f"  saved per-row -> {pred_path}")
    print(f"  saved metrics -> {metrics_path}")

print("\nLocal-only processing complete.")


=== Disease: bphigh (LOCAL) ===


Local RF [bphigh]: 100%|██████████████████████████████████████████████████████████| 1998/1998 [00:12<00:00, 163.96it/s]


  saved per-row -> ./bphigh_local_predictions.csv
  saved metrics -> ./bphigh_local_metrics_tidy.csv

=== Disease: chd (LOCAL) ===


Local RF [chd]: 100%|█████████████████████████████████████████████████████████████| 1998/1998 [00:12<00:00, 163.19it/s]


  saved per-row -> ./chd_local_predictions.csv
  saved metrics -> ./chd_local_metrics_tidy.csv

=== Disease: diabetes (LOCAL) ===


...

  saved per-row -> ./diabetes_local_predictions.csv
  saved metrics -> ./diabetes_local_metrics_tidy.csv

=== Disease: highchol (LOCAL) ===


Local RF [highchol]: 100%|████████████████████████████████████████████████████████| 1998/1998 [00:12<00:00, 161.24it/s]


  saved per-row -> ./highchol_local_predictions.csv
  saved metrics -> ./highchol_local_metrics_tidy.csv

=== Disease: obesity (LOCAL) ===


Local RF [obesity]: 100%|█████████████████████████████████████████████████████████| 1998/1998 [00:12<00:00, 159.24it/s]

  saved per-row -> ./obesity_local_predictions.csv
  saved metrics -> ./obesity_local_metrics_tidy.csv

Local-only processing complete.


In [31]:
import pandas as pd

files = [
    "bphigh_local_metrics_tidy.csv",
    "chd_local_metrics_tidy.csv",
    "diabetes_local_metrics_tidy.csv",
    "highchol_local_metrics_tidy.csv",
    "obesity_local_metrics_tidy.csv"
]

dfs = []
for f in files:
    df = pd.read_csv(f)
    # Add a disease column only if it doesn’t exist
    if "disease" not in df.columns:
        disease = f.split("_")[0]   # file prefix
        df.insert(0, "disease", disease)
    dfs.append(df)

# Concatenate
merged = pd.concat(dfs, ignore_index=True)

# Save
merged.to_csv("all_local_metrics_summary.csv", index=False)
print("Merged file saved -> all_local_metrics_summary.csv")

Merged file saved -> all_local_metrics_summary.csv
